# This notebook has code to create a data sheet containing all the salient information from my todo lists
## It requires a combined document of all monthly todos with a few specific formatting things:
- The code assumes dates go from last to first.  
- Put the year first as a 'subtitle'  
- Put the month next as a 'title'  
- Put the day(s) next as a 'Heading 1'  
- Put each activity done on the day as 'normal'  
- Including a time at the start seperated from the activity description with ": " will allow the code to extract a time for that entry

In [1]:
#Import useful libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import re
from collections import Counter
from docx import Document

In [2]:
#In the interest of making easy to read code
#The below functions do a single action and return something to do add to the dataframe

def make_dataframe_from_dict(rows_dict):
    '''
    Function to make a dataframe, specific for the dataset here.
    requires that:
    1. all keys are lists of the same length
    2. there is a "day" key
    '''
    dataframe_index = [x for x in range(0,len(rows_dict['day']))]
    return pd.DataFrame(rows_dict, columns=rows_dict.keys(), index=dataframe_index)

def concatenate_runs(paragraph):
    '''
    Takes all runs in a paragraph and glues them together
    '''
    temp_string = ''
    for run in paragraph.runs:
        temp_string += str(run.text)
    return temp_string

def is_strikethrough(paragraph):
    '''
    Looks for strikethrough formatting in a paragraph
    Assumes input of a Document.paragraph object
    Returns True if strikethrough appears in any run in paragraph, otherwise returns False
    '''
    for run in paragraph.runs:
        if run.font.strike == True:
            return True
    return False

def get_color(paragraph):
    '''
    Gets each color from the runs and returns them as a list
    useful to change RGB values into colour catergories
    Black = None
    Red = (255, 0, 0)
    Blue = (0, 112, 192)
    Green = (0, 176, 80)
    '''
    colors = []
    for run in paragraph.runs:
        if run.font.color.rgb == (0x00, 0x70, 0xc0):
            colors.append('blue')
        elif run.font.color.rgb == (0xff, 0x00, 0x00):
            colors.append('red')
        elif run.font.color.rgb == None:
            colors.append('black')
        elif run.font.color.rgb == (0x00, 0xb0, 0x50):
            colors.append('green')
        else:
            colors.append(run.font.color.rgb)
    return colors

def get_time_taken(activity_text):
    '''
    Assumes type(activity_text) is str
    looks for ": " signature of time in concatenated run text and returns what came before it as raw string
    '''
    if ": " in activity_text:
        return activity_text.split(": ")[0]
    else:
        return np.nan

In [3]:
#This function uses the functions above to generate a dataframe

def build_df_from_todo(input_doc):
    #this builds a dictionary table with a bunch of columns for various types of formatting
    #uses subtitle to detect year start and end
    rows_dict = {'day':[],'time taken':[], 'activity':[], 'strike':[], 'colors' :[]} 

    #init vars
    day = ''
    month = ''
    year = 'none'
    activity = ''

    for para in input_doc.paragraphs:
        style_flag = para.style.name
        #If style is a title, it is the month
        if style_flag == 'Title':
            month = concatenate_runs(para)
        elif style_flag == 'Subtitle':
            year = concatenate_runs(para)
        #If style is a heading, it is a day of the week
        elif style_flag == 'Heading 1':
            day = concatenate_runs(para) + ' ' + month + ' ' + year
        #If style is normal, it is an activity
        #In this condition we build a row of our table
        elif style_flag == 'Normal':
            activity = concatenate_runs(para)
            rows_dict['day'].append(day)
            rows_dict['activity'].append(activity)
            rows_dict['time taken'].append(get_time_taken(activity))
            rows_dict['strike'].append(is_strikethrough(para))
            rows_dict['colors'].append(get_color(para))
        else:
            print('paragraph style not expected')
    
    return rows_dict

In [4]:
#this code opens the combined text document and then feeds into the build_df_from_todo function to make a useful dataframe

test_doc = Document("/home/sp/Documents/new_todo_parsing/todo_files/todo list.docx")
todo_dict = build_df_from_todo(test_doc)
todo_table = make_dataframe_from_dict(todo_dict)
todo_table.head(5)

,day,time taken,activity,strike,colors
0,Thursday 1st March 2018,9,9: Take histo down to Nick in the store on lvl 1,True,"[black, black, black, black, black]"
1,Thursday 1st March 2018,9-10,9-10: MLI analysis KB22,True,[black]
2,Thursday 1st March 2018,10:30-1:30,10:30-1:30: Animal facilities induction,True,[black]
3,Thursday 1st March 2018,2-3,2-3: Microbiome meeting,True,[black]
4,Thursday 1st March 2018,2:30-5,2:30-5: KB22 Collagen Analysis,True,[black]


In [5]:
#need to drop entry 339 as it messes with later code for datetime
todo_table.loc[339]

day           Monday 18th to Friday 22nd June 2018
time taken                                     NaN
activity                                      Away
strike                                       False
colors                                     [black]
Name: 339, dtype: object

In [6]:
todo_table = todo_table.drop(339)

# Check some things in dataframe

## Remove empty 'activity' entries
These are just empty rows in the input table.

In [7]:
todo_table = todo_table.loc[todo_table['activity'] != '']

## Any non standard colors?
There were but I found and removed them using the below code and then manually changing them to what they should be.  
Yes, (0x5b, 0x9b, 0xd5) and (0x00, 0x00, 0x00)

In [8]:
temp_output = []
for entry_list in todo_table['colors']:
    for item in entry_list: 
        temp_output.append(item)
Counter(temp_output)

Counter({'black': 964,
         'red': 349,
         'green': 85,
         'blue': 811,
         RGBColor(0x00, 0x00, 0x00): 11})

#### Where are they?

In [9]:
def contains_weird_vals(temp_list):
    for item in temp_list:
        if item == (0x5b, 0x9b, 0xd5):
            return True
        elif item == (0x00, 0x00, 0x00):
            return True
    return False

todo_table.loc[todo_table['colors'].map(contains_weird_vals)]

,day,time taken,activity,strike,colors
64,Thursday 22nd March 2018,NaN,KB22 Collagen analysis,True,"[(0, 0, 0)]"
65,Thursday 22nd March 2018,5,5: Double check to make sure old -20C freezer ...,True,"[(0, 0, 0), (0, 0, 0), (0, 0, 0)]"
66,Thursday 22nd March 2018,4-5,4-5: qPCR on Ileum VEGF (repeat) This worked!...,True,"[(0, 0, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0)]"
581,Tuesday 4th September 2018,9-2,9-2: Plasmid prep colony PCR on other colonies...,True,"[black, red, (0, 0, 0), (0, 0, 0)]"
1016,Thursday 10th January 2019,10-11,10-11: Spleen testing of RBCLB today (passed!)...,True,"[(0, 0, 0), blue]"


### Formatting manually fixed other color (was wrong shade of blue) but black still comes up weird sometimes
Below code fixes that

In [10]:
def fix_black(temp_list):
    '''
    Change (0,0,0) to 'black' in todo_table['colors']
    '''
    for item in temp_list:
        if item == (0x00, 0x00, 0x00):
            return ['black']
    return temp_list

todo_table['colors'] = todo_table['colors'].map(lambda x: fix_black(x))

#check that it worked. Should output that we have four colours in todo_table['colors']
temp_output = []
for entry_list in todo_table['colors']:
    for item in entry_list: 
        temp_output.append(item)
Counter(temp_output)

Counter({'black': 968, 'red': 348, 'green': 85, 'blue': 810})

# Lets make seperate columns for each color, each with a boolean output

In [11]:
def contains_color(temp_list, color):
    '''
    Tests if colour is present in list of colours
    '''
    for item in temp_list:
        if item == color:
            return True
    return False

#cycle through colours and make individual columns using map on the colors column repeatedly
for color in ['black', 'red', 'green', 'blue']:
    todo_table.loc[:,color] = todo_table['colors'].map(lambda x: contains_color(x, color))

In [12]:
todo_table

,day,time taken,activity,strike,colors,black,red,green,blue
0,Thursday 1st March 2018,9,9: Take histo down to Nick in the store on lvl 1,True,"[black, black, black, black, black]",True,False,False,False
1,Thursday 1st March 2018,9-10,9-10: MLI analysis KB22,True,[black],True,False,False,False
2,Thursday 1st March 2018,10:30-1:30,10:30-1:30: Animal facilities induction,True,[black],True,False,False,False
3,Thursday 1st March 2018,2-3,2-3: Microbiome meeting,True,[black],True,False,False,False
4,Thursday 1st March 2018,2:30-5,2:30-5: KB22 Collagen Analysis,True,[black],True,False,False,False
...,...,...,...,...,...,...,...,...,...
1437,Thursday 30th May 2019,NaN,Deal with RNA extraction problem,False,[red],False,True,False,False
1438,Thursday 30th May 2019,3-5,3-5: TJ16 CAE Mast cell hunting,False,[red],False,True,False,False
1439,Friday 31st May 2019,NaN,[sick],False,[black],True,False,False,False
1440,Friday 31st May 2019,9-10,9-10: Freezer move into loaned -80C. With Shan...,False,"[red, red, red]",False,True,False,False


## Next cell checks to make sure the number of entries with a given color match between the new boolean columns and the original 'colors' column
Can't just use Counter() here since it does not like lists. I feel I could do this more elegantly but frankly it doesn't need to be perfect.

In [13]:
def def_in_colors(pandas_series, color):
    '''
    Counts number of entries with 
    '''
    counter = 0
    for entry_list in pandas_series:
        for item in entry_list:
            if item == color:
                counter += 1
                break
    return counter

for color in ['black', 'red', 'green', 'blue']:
    in_col = len(todo_table.loc[todo_table[color] == True])
    in_colors = def_in_colors(todo_table['colors'], color)
    print(color, in_col, in_colors)

black 631 631
red 236 236
green 56 56
blue 578 578


## Now we are confident it worked, we can drop the superfluous 'colors' col

In [14]:
todo_table = todo_table.drop(index=1,columns=['colors'])
todo_table.head(3)

,day,time taken,activity,strike,black,red,green,blue
0,Thursday 1st March 2018,9,9: Take histo down to Nick in the store on lvl 1,True,True,False,False,False
2,Thursday 1st March 2018,10:30-1:30,10:30-1:30: Animal facilities induction,True,True,False,False,False
3,Thursday 1st March 2018,2-3,2-3: Microbiome meeting,True,True,False,False,False


# Lets check for conflicts between formatting options
Since I sometimes would combine multiple items on a single line, there can be e.g strikethrough (finished activity) and red (planned, but not done) on the same line even though those two things contradict each other  
Only line 297 doesn't have black text, and it has blue instead (parts of this were done, parts unplanned, and some not done)  
I will treat these as "partly done" as if they were green text. This will bias partly done upwards by 13 events.

In [15]:
todo_table.loc[todo_table['strike'] == True].loc[todo_table['red'] == True]

,day,time taken,activity,strike,black,red,green,blue
15,Monday 5th March 2018,2-5,2-5: MLI + Collagen images (Gang),True,True,True,False,False
164,Friday 20th April 2018,NaN,"Order primers, double check they tick all the ...",True,True,True,False,False
233,Wednesday 16th May 2018,9-10,"9-10: Move freezer stuff, and fridge stuff Fre...",True,True,True,False,True
297,Thursday 7th June 2018,NaN,"Sort out orders, set up excel sheet, check low...",True,False,True,False,True
344,Monday 25th June 2018,3-4,"3-4: Organise for rest of week, analyse PCR data",True,True,True,False,False
485,Monday 6th August 2018,2-5,2-5: TJ16 RNA extraction pt 4 Anti-PCP ELISA c...,True,True,True,False,True
502,Monday 13th August 2018,1-5,1-5: Gang; PCR of hTryptase 3’ UTR fragments f...,True,True,True,False,False
540,Wednesday 22nd August 2018,1-5,1-5: NH09 BALF counts (TJ) TJ19 RNA extraction...,True,True,True,False,True
543,Thursday 23rd August 2018,9-1,9-1: TJ19 RNA extraction 3dpi pt 2 NH09 BALF c...,True,True,True,False,True
545,Thursday 23rd August 2018,2-5,2-5: RT-PCR for ethics variation (Gang) Transf...,True,True,True,False,False


# Square brackets are a flag for a note I made, lets find those
I will mark them as notes in an appropiate col  
There is a few, so I will only print a few here

In [16]:
todo_table.loc[todo_table['activity'].map(lambda x: '[' in x) == True].head(5)

,day,time taken,activity,strike,black,red,green,blue
89,Wednesday 28th March 2018,NaN,[PCR for Shakti],False,False,True,False,False
107,Friday 6th April 2018,NaN,"[Bereavement leave, funeral]",False,True,False,False,False
131,Friday 13th April 2018,NaN,[Sick again],False,True,False,False,False
473,Wednesday 1st August 2018,NaN,[1 hour of time in lieu],False,True,False,False,False
532,Monday 20th August 2018,NaN,[30 mins time in lieu],False,True,False,False,False


## Unmarked notes..
There are a few notes I did not mark with square bracket. Most are sick notes which are easy to find:

In [17]:
todo_table.loc[todo_table['activity'].map(lambda x: 'sick' in x) == True]

,day,time taken,activity,strike,black,red,green,blue
20,Wednesday 7th March 2018,NaN,Went home sick at 11am,False,True,False,False,False
108,Monday 9th April 2018,NaN,Was sick.,False,True,False,False,False
280,Monday 4th June 2018,NaN,Was sick.,False,True,False,False,False
664,Tuesday 25th September 2018,NaN,[Was sick],False,True,False,False,False
675,Wednesday 26th September 2018,9-5,9-5: IFN-lambda ELISA RV017 BALF didn’t get pl...,False,True,True,False,False
748,Thursday 25th October 2018,NaN,"[went home sick at 12, took time in lieu]",False,True,False,False,False
848,Monday 19th November 2018,NaN,[went home sick at 3:30],False,True,False,False,False
891,Wednesday 28th November 2018,NaN,[sick],False,True,False,False,False
1170,Monday 25th February 2019,NaN,[went home sick @ 1pm],False,True,False,False,False
1185,Thursday 28th February 2019,NaN,[went home sick],False,True,False,False,False


#### 3 entries without brackets, one other entry that looks interesting, what does activity descp. actually say?
Guess it just counts as "not done" and will be flagged as such with red text formatting.  
Notably the sick day it refers to is in this list!

In [18]:
todo_table.loc[675,'activity']

'9-5: IFN-lambda ELISA RV017 BALF didn’t get plate coated because I was sick'

# Need to convert text dates into actual computer dates
### This will make it a lot easier for certain stats and visualisations

In [19]:
#First, there are some double spaces so lets fix that...
todo_table['day'] = todo_table['day'].map(lambda x: x.replace('  ',' '))

In [20]:
#check all entries have same number of words - All good!
Counter(todo_table['day'].map(lambda x: len(x.split(' '))))

Counter({4: 1413})

In [21]:
todo_table

,day,time taken,activity,strike,black,red,green,blue
0,Thursday 1st March 2018,9,9: Take histo down to Nick in the store on lvl 1,True,True,False,False,False
2,Thursday 1st March 2018,10:30-1:30,10:30-1:30: Animal facilities induction,True,True,False,False,False
3,Thursday 1st March 2018,2-3,2-3: Microbiome meeting,True,True,False,False,False
4,Thursday 1st March 2018,2:30-5,2:30-5: KB22 Collagen Analysis,True,True,False,False,False
5,Friday 2nd March 2018,9-4,9-4: BALF processing for endpoint (Alex),True,True,False,False,False
...,...,...,...,...,...,...,...,...
1437,Thursday 30th May 2019,NaN,Deal with RNA extraction problem,False,False,True,False,False
1438,Thursday 30th May 2019,3-5,3-5: TJ16 CAE Mast cell hunting,False,False,True,False,False
1439,Friday 31st May 2019,NaN,[sick],False,True,False,False,False
1440,Friday 31st May 2019,9-10,9-10: Freezer move into loaned -80C. With Shan...,False,False,True,False,False


In [22]:
def strip_date_suffixes(daytext):
    '''
    Remove string date suffixes that get in the way of the formatter.
    '''
    banned = 'st,nd,rd,th,.'.split(',')
    for ban in banned:
        daytext = daytext.replace(ban,'')
    return daytext

def convert_datetime(date_text):
    month_dict = {'january': 1, 'february': 2, 'march': 3, 'april': 4, 'may': 5, 'june': 6, 'july': 7, 
                  'august': 8, 'september': 9, 'october': 10, 'november': 11, 'december': 12}
    date_text = date_text.split(' ')
    day = int(strip_date_suffixes(date_text[1]))
    month = int(month_dict[date_text[2].lower()])
    year = int(date_text[3])
    return datetime.date(year,month,day)

todo_table.loc[:,'date'] = todo_table['day'].map(convert_datetime)
#and a ordinal time for ease of use...
todo_table.loc[:,'ordinal_date'] = todo_table['date'].map(lambda x: x.toordinal()) 
#minus the smallest value to get cleaner numbers
min_ordinal = np.min(todo_table['ordinal_date'])
todo_table['ordinal_date'] = todo_table['ordinal_date'].map(lambda x: x - min_ordinal)

In [23]:
todo_table.head(3)

,day,time taken,activity,strike,black,red,green,blue,date,ordinal_date
0,Thursday 1st March 2018,9,9: Take histo down to Nick in the store on lvl 1,True,True,False,False,False,2018-03-01,0
2,Thursday 1st March 2018,10:30-1:30,10:30-1:30: Animal facilities induction,True,True,False,False,False,2018-03-01,0
3,Thursday 1st March 2018,2-3,2-3: Microbiome meeting,True,True,False,False,False,2018-03-01,0


# For my final act(s):
### I will make an "activity status" col that is built from every combo of formatting in strike to blue cols
There will be a priority list based on maximum caution, in order:  
0. activity contains either "sick" or "\[" = 'note'
1. strike + red = 'partly done'
2. red + green = 'partly done'
2. red = 'not done'
3. green = 'partly done'
4. blue = 'done'
5. strike = 'done'
6. black = 'done'

In [24]:
def activity_status_finder(df_entry):
    if "sick" in df_entry['activity']:
        return 'note'
    elif "[" in df_entry['activity']:
        return 'note'
    elif df_entry['red'] == True:
        if df_entry['strike'] == True:
            return 'partly done'
        elif df_entry['green'] == True:
            return 'partly done'
        else:
            return 'not done'
    elif df_entry['green'] == True:
        return 'partly done'
    elif df_entry['blue'] == True:
        return 'done'
    elif df_entry['strike'] == True:
        return 'done'
    elif df_entry['black'] == True:
        return 'done'
    else:
        return 'unknown'

temp_series = pd.Series()
for i in todo_table.index:
    temp_series.loc[i] = activity_status_finder(todo_table.loc[i,:])
    
todo_table.loc[:,'activity status'] = temp_series

/home/sp/miniconda3/envs/python37/lib/python3.7/site-packages/ipykernel_launcher.py:24: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


## Setting up a timedelta col for each activity
First, lets build a series and clean up some data  
This code does not contain things that had only a start time, which is okay since I am more interested in building bar graphs based on start-end times.

In [25]:
temp_series = todo_table["time taken"].loc[todo_table["time taken"].map(lambda x: "-" in str(x))]
temp_series = temp_series.map(lambda x: x.split('-'))
#this line drops anything that has more or less than 2 values
temp_series = temp_series.drop(temp_series.loc[temp_series.map(lambda x: len(x)) != 2].index)
#drop if contains certain words
temp_series = temp_series.drop(temp_series.loc[temp_series.map(lambda x: "Fill" in x)].index)
temp_series = temp_series.drop(temp_series.loc[temp_series.map(lambda x: "In" in x)].index)
temp_series = temp_series.drop(temp_series.loc[temp_series.map(lambda x: 'and' in x[1])].index)
temp_series = temp_series.drop(temp_series.loc[temp_series.map(lambda x: '(' in x[1])].index)

### Checking for weird things in my time column  
This one liner looks only at the first number (the start time) and just drops anything with a colon in it. It then turns the numbers into int dtypes and returns a count of each number.

In [26]:
#wrap in print func to reduce whitespace
print(Counter(temp_series.map(lambda x: x[0]).drop(temp_series.loc[temp_series.map(lambda x: ":" in x[0])].index).map(lambda x: int(x))))

Counter({9: 291, 2: 85, 3: 78, 1: 71, 10: 64, 4: 46, 11: 34, 12: 32, 8: 7, 7: 4, 6: 2})


It is reasonable to assume anything <=5 and >=9 is a morning number but the same cannot be said for values <9 which could be an early start or a late nighter  
There is only a few entries before 9am so lets look at those in the full dataset.  
Although likely not clear to another reader, my personal experience with all of these entries is that they are indeed morning activities.

In [27]:
todo_table.loc[temp_series.loc[temp_series.map(lambda x: '6' in x[0] or '7' in x[0] or '8' in x[0])].index].loc[:,['time taken','activity']]

,time taken,activity
36,8-10,8-10: 20x and 40x micrographs of Ileum samples...
340,8-4,8-4: PISA – cancelled this morning
341,8-12,8-12: Check PBS and RBCLB levels – make up RBC...
432,8:30-5:30,8:30-5:30: PISA056 S-C
457,7-2,7-2: Colon macrophage isolation trial #1 can’t...
486,8:30-5,8:30-5: PISA045 (blood ready at 9am)
522,6-2,6-2: Macrophage project cell isolation
533,6-2,6-2: Macrophage project/SS03 immune cell isola...
593,8:30-5,8:30-5: PISA073 M-P
631,8:30-9:30,8:30-9:30: Safety and lab meetings


End times at 8pm or 9pm might actually be morning times, lets check that by eye:  
Conclusion: Can safely assume all '8's are pm and all '9's are am

In [28]:
todo_table.loc[temp_series.loc[temp_series.map(lambda x: '8' in x[1] or '9' in x[1])].index].loc[:,['time taken','activity']]

,time taken,activity
109,9-9:10,9-9:10: Check on old bacto lab -20C.
472,11-8,11-8: PISA066 B-C blood ready at 11:30am Cance...
495,11:30-8,11:30-8: PISA064 RJC (Blood ready at 12pm) can...
616,11:30-8,11:30-8: PISA067 K-C (blood ready at 12) postp...
631,8:30-9:30,8:30-9:30: Safety and lab meetings
639,11:30-8,11:30-8: PISA067 K-C (blood ready at 12)
761,9-9:30,9-9:30: lab meeting – no lab meeting
851,9-9:30,9-9:30: PA01 O/N in 2.5uM test or equivalent e...
1199,9-9:30,9-9:30: Check battery on west wing electronic ...
1231,8-9,8-9: Mice bedding swaps for air and smk


Now we have a good idea of what numbers correspond to morning or afternoon, we can convert to datetime.time objects

In [29]:
def convert_to_time(time_string, start_time = True):
    """Converts the strings in temp_series into datetime.time objects. For endtimes, set start_time flag to False
    which makes different assumptions about the time"""
    if start_time == True:
        if ":" in time_string:
            time_string = time_string.split(":")
            #morning
            if int(time_string[0]) <= 5:
                #make pm time
                temp_time = int(time_string[0]) + 12
                return datetime.time(temp_time,int(time_string[1]))

            #afternoon
            elif int(time_string[0]) >= 6:
                return datetime.time(int(time_string[0]),int(time_string[1]))
        else:
            time_string = int(time_string)
            #morning
            if time_string <= 5:
                #make pm time
                time_string += 12
                return datetime.time(time_string)

            #afternoon
            elif int(time_string) >= 6:
                return datetime.time(time_string)
    
    #end times
    else:
        if ":" in time_string:
            time_string = time_string.split(":")
            #morning
            if int(time_string[0]) < 9:
                #make pm time
                temp_time = int(time_string[0]) + 12
                return datetime.time(temp_time,int(time_string[1]))

            #afternoon
            elif int(time_string[0]) >= 9:
                return datetime.time(int(time_string[0]),int(time_string[1]))
        else:
            time_string = int(time_string)
            #morning
            if time_string < 9:
                #make pm time
                time_string += 12
                return datetime.time(time_string)

            #afternoon
            elif int(time_string) >= 9:
                return datetime.time(time_string)

In [30]:
#make series of tuples with start and finish times
start_end_times = temp_series.map(lambda x: (convert_to_time(x[0]),convert_to_time(x[1],start_time=False)))

In [31]:
start_only = temp_series.map(lambda x: (convert_to_time(x[0])))
end_only = temp_series.map(lambda x: (convert_to_time(x[1],start_time=False)))

### Need to add just start times
For some events there is a start time but no end time

In [32]:
#these three lines select just those with a single number
temp_df = todo_table['time taken'].dropna()
temp_df = temp_df.loc[temp_df.map(lambda x: len(x) <= 3)]
temp_df = temp_df.loc[temp_df.map(lambda x: "-" not in x)]
#remove text
temp_df = temp_df.map(lambda x: x.replace("pm","").replace("[",""))
#convert to time
temp_df = temp_df.map(lambda x: datetime.time(int(x)))

#combine with other start times
start_only = start_only.combine_first(temp_df)

In [33]:
todo_table.loc[:,'start_time'] = start_only
todo_table.loc[:,'end_time'] = end_only
todo_table.head(2)

,day,time taken,activity,strike,black,red,green,blue,date,ordinal_date,activity status,start_time,end_time
0,Thursday 1st March 2018,9,9: Take histo down to Nick in the store on lvl 1,True,True,False,False,False,2018-03-01,0,done,09:00:00,NaN
2,Thursday 1st March 2018,10:30-1:30,10:30-1:30: Animal facilities induction,True,True,False,False,False,2018-03-01,0,done,10:30:00,13:30:00


### Adding a time length for events with start and end times
This is currently in type int and assumes minutes

In [34]:
def dtt2timestamp(dtt):
    ts = (dtt.hour * 60 + dtt.minute) * 60 + dtt.second
    return ts
#int((dtt2timestamp(todo_table['end_time'][2])-dtt2timestamp(todo_table['start_time'][2]))/60)
def time_difference(start,end):
    return int((dtt2timestamp(end)-dtt2timestamp(start))/60)

In [35]:
#select all events with both a start and end time
temp_df = todo_table[["start_time","end_time"]].loc[
    todo_table[["start_time","end_time"]].isna().apply(
    lambda x: x['start_time'] == x['end_time'] == False,axis=1)]
#generate series with minutes taken from start to finish
temp_df = temp_df.apply(lambda x: time_difference(x['start_time'],x['end_time']),axis=1)
#create time_taken in todo_table
todo_table['time_taken'] = temp_df

## Make a "planned" vs not planned col
Only blue entries were unplanned, and then done. If they were unplanned and not done, they simply weren't left as an entry

In [36]:
def invert_bool(bool_entry):
    if bool_entry == True:
        return False
    else:
        return True
todo_table.loc[:,'planned'] = todo_table['blue'].map(invert_bool)

In [37]:
todo_table.head(2)

,day,time taken,activity,strike,black,red,green,blue,date,ordinal_date,activity status,start_time,end_time,time_taken,planned
0,Thursday 1st March 2018,9,9: Take histo down to Nick in the store on lvl 1,True,True,False,False,False,2018-03-01,0,done,09:00:00,NaN,NaN,True
2,Thursday 1st March 2018,10:30-1:30,10:30-1:30: Animal facilities induction,True,True,False,False,False,2018-03-01,0,done,10:30:00,13:30:00,180.0,True


# Create a day of the week column
First check what comes up in the "day" col

In [38]:
Counter(todo_table['day'].map(lambda x: x.split(' ')[0]))

Counter({'Thursday': 281,
         'Friday': 267,
         'Monday': 297,
         'Tuesday': 292,
         'Wednesday': 274,
         'Sunday': 2})

In [39]:
#create "dayofweek" col
todo_table['dayofweek'] = todo_table['day'].map(lambda x: x.split(' ')[0])

## Save files...
First saves the file as is  
Then drops formatting cols and saves again as final, as both csv and excel

In [40]:
#reorganise columns for readability
new_col_order = ['date', 'ordinal_date', 'dayofweek', 'day', 'time taken', 'start_time','end_time', 
                 'time_taken', 'activity', 'strike', 'black', 'red', 'green', 'blue', 
                 'activity status', 'planned']
todo_table = todo_table.reindex(new_col_order,axis=1)

In [41]:
todo_table.to_csv('parsed todo table with formatting cols.csv')
todo_format = todo_table.drop(columns=['strike','black','red','blue', 'green', 'time taken'])
todo_format.to_csv('parsed todo table.csv')
todo_format.to_excel('parsed todo table.xlsx')

In [42]:
todo_format.head(5)

,date,ordinal_date,dayofweek,day,start_time,end_time,time_taken,activity,activity status,planned
0,2018-03-01,0,Thursday,Thursday 1st March 2018,09:00:00,NaN,NaN,9: Take histo down to Nick in the store on lvl 1,done,True
2,2018-03-01,0,Thursday,Thursday 1st March 2018,10:30:00,13:30:00,180.0,10:30-1:30: Animal facilities induction,done,True
3,2018-03-01,0,Thursday,Thursday 1st March 2018,14:00:00,15:00:00,60.0,2-3: Microbiome meeting,done,True
4,2018-03-01,0,Thursday,Thursday 1st March 2018,14:30:00,17:00:00,150.0,2:30-5: KB22 Collagen Analysis,done,True
5,2018-03-02,1,Friday,Friday 2nd March 2018,09:00:00,16:00:00,420.0,9-4: BALF processing for endpoint (Alex),done,True
